In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import bs4
import re
import os

In [14]:
%load_ext autoreload
%autoreload 2

from sofa_score_scrap import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Players Stats 

In [3]:
# s.get('https://www.sofascore.cddom/pt/atletico-mineiro-internacional/qOsCO', headers={'x-test2': 'true'})
resp = requests.get("https://www.sofascore.com/event/7724681/statistics/players/json?_=154328071")

In [4]:
players_df = pd.DataFrame(resp.json()["players"])

In [5]:
get_per_player_data(players_df)

,name,slug,shortName,id,national,team_name,team_id,goals,goalAssist,totalTackle,...,totalCross,totalLongBalls,dispossessed,wasFouled,fouls,saves,punches,runsOut,goodHighClaim,team
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,Juan Cazares,atletico-mineiro,J. Cazares,158411,False,Atlético Mineiro,1977,1,1,0,...,8 (2),2 (2),2,1,2,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Andrés D'Alessandro,internacional,A. D'Alessandro,17639,False,Internacional,1966,1,,1,...,12 (3),5 (2),3,3,1,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Víctor Cuesta,internacional,V. Cuesta,587148,False,Internacional,1966,,,1,...,0 (0),11 (7),0,3,2,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,David Terans,atletico-mineiro,D. Terans,586662,False,Atlético Mineiro,1977,1,,1,...,1 (0),2 (1),0,0,1,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Nico López,internacional,N. López,255881,False,Internacional,1966,,,0,...,5 (2),7 (4),1,1,1,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Patric,atletico-mineiro,Patric,78853,False,Atlético Mineiro,1977,,,4,...,2 (0),8 (1),2,1,1,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Luan,atletico-mineiro,Luan,243427,False,Atlético Mineiro,1977,,,4,...,2 (0),1 (0),0,1,4,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Emerson Santos,internacional,E. Santos,844015,False,Internacional,1966,,,2,...,0 (0),4 (2),1,0,0,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Fabiano,internacional,F. Leismann,551304,False,Internacional,1966,,,2,...,6 (0),5 (2),0,0,2,NaN,NaN,NaN,NaN,INT


### Game Stats

In [335]:
resp = requests.get("https://www.sofascore.com/event/7724681/general/json")

In [336]:
resp

<Response [200]>

In [337]:
pd.DataFrame(game_statistics(resp), index=[0]).set_index("game")

,team_away,team_home,data,hora,away_score,home_score,result,Ball possession_home,Ball possession_away,Total shots_home,...,total_goals_Under_3.5,fl_total_goals_Under_3.5,total_goals_Over_4.5,fl_total_goals_Over_4.5,total_goals_Under_4.5,fl_total_goals_Under_4.5,total_goals_Over_5.5,fl_total_goals_Over_5.5,total_goals_Under_5.5,fl_total_goals_Under_5.5
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,CAM,INT,2018-11-21,19:30:00,2,1,-1,54%,46%,15,...,1.22,True,9.00,False,1.07,True,19.00,False,1.02,True


### Pega links dos jogos históricos

#### Pega os IDs das temporadas

In [ ]:
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325")
c = bs4.BeautifulSoup(comp.content)

In [204]:
temporadas = c.find_all("a", attrs={"class":"pointer js-uniqueTournament-page-load-season"})

id_temporada = [(a.text.strip(), a.attrs["data-season-id"]) for a in temporadas]
id_temporada

[('2018', '16183'),
 ('2017', '13100'),
 ('2016', '11429'),
 ('2015', '10173'),
 ('2014', '7778'),
 ('2013', '6075'),
 ('2012', '4438'),
 ('2011', '3311'),
 ('2010', '2684'),
 ('2009', '2079'),
 ('2008', '1223')]

#### Para cada Rodada, pega os dados com os links para os jogos

In [356]:
def get_info_rodada(resp):
    games = resp.json()['roundMatches']["tournaments"][0]['events']
    
    resp = {
        "game":[],
        "fixture": [],
        "id": [],
        "team_home":[],
        "team_away":[],
        "date":[],
        "link":[],
        "home_score":[],
        "away_score":[]
    }
    
    for game in games:
        try:        
            resp["home_score"].append(game['homeScore']["current"])
            resp["away_score"].append(game['awayScore']["current"])        
        except TypeError:
            #resp["home_score"].append(-1)
            #resp["away_score"].append(-1)
            continue
        
        resp["fixture"].append(game['roundInfo']["round"])
        resp["id"].append(game["id"])
        resp["team_home"].append(de_para_siglas.loc[game["homeTeam"]["name"]].iloc[0])
        resp["team_away"].append(de_para_siglas.loc[game["awayTeam"]["name"]].iloc[0])
        
        resp["date"].append(game["formatedStartDate"].replace(".", "-")[:-1])
        resp["link"].append("https://www.sofascore.com/pt/" + game["slug"] + "/" + game["customId"])
        resp["game"].append(resp["team_home"][-1] + " X " + resp["team_away"][-1] + " " + resp["date"][-1])
    
    return(pd.DataFrame(resp).set_index("game"))

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
    "upgrade-insecure-requests": '1',
    "Connection": "close",    
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "max-age=0"
}
resp = requests.get("https://www.sofascore.com/u-tournament/325/season/16183/matches/round/3", headers=headers)
resp.encoding = 'UTF-8'

In [319]:
s = requests.Session()
s.get('https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325')

<Response [200]>

In [5]:
de_para_siglas = pd.read_excel("de_para_siglas_br.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

In [6]:
import sofa_score_scrap

In [358]:
s = requests.Session()
s.get('https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325')

<Response [200]>

In [359]:
for temp in id_temporada:
    ret = []
    
    for rodada in range(1, 39):
        print(temp[0] + ": " + str(rodada), end="\r")
        resp = s.get("https://www.sofascore.com/u-tournament/325/season/"+ temp[1] + "/matches/round/" + str(rodada))
        resp.encoding = 'UTF-8'
        ret.append(get_info_rodada(resp))
    
    ret = pd.concat(ret)
    ret.to_csv("links_sofa_score/sofa_score_links_BR_"+temp[0]+".csv", sep=";")

### Pega informações dos jogos históricos

In [363]:
base = pd.read_csv("links_sofa_score/sofa_score_links_BR_2018.csv", sep=";")

In [53]:
s = requests.Session()
s.get('https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325')

<Response [200]>

In [51]:
list(reversed(os.listdir("links_sofa_score/")[7:-3]))

['sofa_score_links_BR_2015.csv']

In [54]:
for file in reversed(os.listdir("links_sofa_score/")[7:-3]):
    base = pd.read_csv("links_sofa_score/" + file, sep=";")
    
    resp_game = []
    resp_players = []
    
    name = file[-11:-4]
    
    i = 0
    for game_id in base.id:
        print(name + " " + str(i), end="\r")
        resp = s.get("https://www.sofascore.com/event/" + str(game_id) + "/general/json")
        
        resp2 = s.get("https://www.sofascore.com/event/" + str(game_id) + "/statistics/players/json")
        try:
            players_df = pd.DataFrame(resp2.json()["players"])
        except KeyError:
            resp_game.append(pd.DataFrame(game_statistics(resp, players_df), index=[0]).set_index("game"))
            continue
        
        resp_game.append(pd.DataFrame(game_statistics(resp, players_df), index=[0]).set_index("game"))
        resp_players.append(get_per_player_data(players_df)) 
        i = i + 1
    
    ret_game = pd.concat(resp_game)
    ret_game.to_csv("BR_data_sofa_score/game_data_"+ name +".csv", sep=";")
    
    ret_players = pd.concat(resp_players)
    ret_players.to_csv("BR_data_sofa_score/players_data_"+ name +".csv", sep=";")

TypeError: 'NoneType' object is not subscriptable